In [3]:
import argparse
import os
import pickle
import random
import time
import sys
import datetime
import copy
import numpy as np
import chainer
from chainer import optimizers
from chainer import cuda
from chainer import serializers
import tensorflow as tf
import multiprocessing as mp

sys.path.append('../../chainer-SeqGAN/')

from models import SeqGAN, TextCNN, SeqEncoder
from optimizer_hook import NamedWeightDecay

parser = argparse.ArgumentParser()
parser.add_argument("--out", default='')
parser.add_argument("--gen", default='')
parser.add_argument("--dis", default='')
parser.add_argument('--gpu', '-g', type=int, default=0)
parser.add_argument('--parallel', '-p', default=0, type=int)

#  Generator  Hyper-parameters
parser.add_argument("--gen_emb_dim", type=int, default=128)
parser.add_argument("--gen_hidden_dim", type=int, default=128)
parser.add_argument("--gen_grad_clip", type=int, default=5)
parser.add_argument("--gen_lr", type=float, default=1e-3)
parser.add_argument("--num_lstm_layer", type=int, default=1)
parser.add_argument("--no-dropout", dest='dropout', action='store_false', default=True)

#  Discriminator  Hyper-parameters
parser.add_argument("--dis_embedding_dim", type=int, default=64)
parser.add_argument("--dis_filter_sizes", default="1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20")
parser.add_argument("--dis_num_filters", default="100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160")
parser.add_argument("--dis_dropout_keep_prob", type=float, default=0.75)
parser.add_argument("--dis_l2_reg_lambda", type=float, default=0.2)
parser.add_argument("--dis_lr", type=float, default=1e-4)

#  Training  Hyper-parameters
parser.add_argument("--batch_size", type=int, default=100)
parser.add_argument("--total_epoch", type=int, default=800)
parser.add_argument("--gen_pretrain_epoch", type=int, default=100)
parser.add_argument("--dis_pretrain_epoch", type=int, default=50)

parser.add_argument("--g_steps", type=int, default=1)
parser.add_argument("--d_steps", type=int, default=5)
parser.add_argument("--K", type=int, default=5)

parser.add_argument("--rollout_update_ratio", type=float, default=0.8)
parser.add_argument("--sample_per_iter", type=int, default=10000)

parser.add_argument("--free-pretrain", dest='free_pretrain', action='store_true', default=False)
parser.add_argument("--ae-pretrain", dest='ae_pretrain', action='store_true', default=False)

args, _ = parser.parse_known_args()


In [4]:
# load nico-comment dataset loader
with open('nico_comment_processed.dat', 'rb') as f:
    train_comment_data, test_comment_data, train_tag_data, test_tag_data, vocab, tag_id = pickle.load(f)

train_num = len(train_comment_data)
test_num = len(test_comment_data)
vocab_size = 3000
seq_length = 30
start_token = 0

def progress_report(count, start_time, batchsize):
    duration = time.time() - start_time
    throughput = count * batchsize / duration
    sys.stderr.write(
        '\r{} updates ({} samples) time: {} ({:.2f} samples/sec)'.format(
            count, count * batchsize, str(datetime.timedelta(seconds=duration)).split('.')[0], throughput
        )
    )


In [7]:
generator = SeqGAN(vocab_size=vocab_size, emb_dim=args.gen_emb_dim, hidden_dim=args.gen_hidden_dim,
                   sequence_length=seq_length, start_token=start_token, lstm_layer=1,
                   dropout=args.dropout, free_pretrain=args.free_pretrain, encoder=None).to_gpu()

serializers.load_hdf5('runs/first/models/gen_pretrain_20.model', generator)

In [2]:
discriminator = TextCNN(num_classes=2, vocab_size=vocab_size, embedding_size=args.dis_embedding_dim,
                        filter_sizes=[int(n) for n in args.dis_filter_sizes.split(',')],
                        num_filters=[int(n) for n in args.dis_num_filters.split(',')]
                        ).to_gpu()

NameError: name 'TextCNN' is not defined

In [16]:
test_loss = []
batch_size = 100
perm = np.random.permutation(test_num)
for i in range(0, test_num, batch_size):
    batch = test_comment_data[perm[i:i + batch_size]]
    g_loss = generator.pretrain_step(batch)
    test_loss.append(float(g_loss.data))
print('\npre-trained test_loss {}'.format(np.mean(test_loss)))
test_count = args.gen_pretrain_epoch



pre-trained test_loss 1.4349755005041758


In [20]:
positive = train_comment_data[np.random.permutation(train_num)[:args.sample_per_iter]]


In [21]:
len(positive)

10000

In [22]:
%prun negative = generator.generate(args.sample_per_iter)


In [24]:
positive.shape

(10000, 30)

In [25]:
positive[0]

array([2148,  928,  362, 1158,  359,  358,  351,  347,  389, 1340, 1751,
        358,  331,  358,  391,  326,    1, 2999, 2999, 2999, 2999, 2999,
       2999, 2999, 2999, 2999, 2999, 2999, 2999, 2999], dtype=int32)